In [63]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel.data import PanelData
from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt

In [64]:
df=pd.read_csv("../data/merged/len4_ndb_agg_in_oral_DropNever.csv",encoding="shift-jis",index_col=0)

In [65]:
## processing
df_columns = df.columns # 列名をリストとして取得
import re

# 正規表現パターンを作成します
pattern = r'id|^elasped_(m[0-5]|[0-9]|1\d|20)$'

# 列名のリストを作成します
column_names = [col for col in df_columns if re.search(pattern, col)]

# 列名のリストを表示します
print(column_names)
rest_columns=["year","generic_per","後発品区分"]
column_names=rest_columns+column_names
column_names

['id', 'id_l4', 'elasped_m5', 'elasped_m4', 'elasped_m3', 'elasped_m2', 'elasped_m1', 'elasped_0', 'elasped_1', 'elasped_2', 'elasped_3', 'elasped_4', 'elasped_5', 'elasped_6', 'elasped_7', 'elasped_8', 'elasped_9', 'elasped_10', 'elasped_11', 'elasped_12', 'elasped_13', 'elasped_14', 'elasped_15', 'elasped_16', 'elasped_17', 'elasped_18', 'elasped_19', 'elasped_20', 'id_1145.0', 'id_1147.0', 'id_1149.0', 'id_1219.0', 'id_1231.0', 'id_1242.0', 'id_1249.0', 'id_1319.0', 'id_1324.0', 'id_1329.0', 'id_2119.0', 'id_2189.0', 'id_2229.0', 'id_2234.0', 'id_2239.0', 'id_2249.0', 'id_2259.0', 'id_2290.0', 'id_2319.0', 'id_2325.0', 'id_2329.0', 'id_2359.0', 'id_2399.0', 'id_2454.0', 'id_2459.0', 'id_2529.0', 'id_2590.0', 'id_2646.0', 'id_2649.0', 'id_2655.0', 'id_2659.0', 'id_3136.0', 'id_3214.0', 'id_3327.0', 'id_3399.0', 'id_3919.0', 'id_3992.0', 'id_3999.0', 'id_4413.0', 'id_4490.0', 'id_6250.0', 'id_6290.0', 'id_7990.0']


['year',
 'generic_per',
 '後発品区分',
 'id',
 'id_l4',
 'elasped_m5',
 'elasped_m4',
 'elasped_m3',
 'elasped_m2',
 'elasped_m1',
 'elasped_0',
 'elasped_1',
 'elasped_2',
 'elasped_3',
 'elasped_4',
 'elasped_5',
 'elasped_6',
 'elasped_7',
 'elasped_8',
 'elasped_9',
 'elasped_10',
 'elasped_11',
 'elasped_12',
 'elasped_13',
 'elasped_14',
 'elasped_15',
 'elasped_16',
 'elasped_17',
 'elasped_18',
 'elasped_19',
 'elasped_20',
 'id_1145.0',
 'id_1147.0',
 'id_1149.0',
 'id_1219.0',
 'id_1231.0',
 'id_1242.0',
 'id_1249.0',
 'id_1319.0',
 'id_1324.0',
 'id_1329.0',
 'id_2119.0',
 'id_2189.0',
 'id_2229.0',
 'id_2234.0',
 'id_2239.0',
 'id_2249.0',
 'id_2259.0',
 'id_2290.0',
 'id_2319.0',
 'id_2325.0',
 'id_2329.0',
 'id_2359.0',
 'id_2399.0',
 'id_2454.0',
 'id_2459.0',
 'id_2529.0',
 'id_2590.0',
 'id_2646.0',
 'id_2649.0',
 'id_2655.0',
 'id_2659.0',
 'id_3136.0',
 'id_3214.0',
 'id_3327.0',
 'id_3399.0',
 'id_3919.0',
 'id_3992.0',
 'id_3999.0',
 'id_4413.0',
 'id_4490.0',
 'id_625

In [66]:
df["year2"]=df["year"]-2014
df["log_quantity"]=np.log(df["総計"]+0.001)
df["log_revenue"]=np.log(df["revenue"]+0.001)
df_generic=df.copy()
df_branded=df.copy()
df_branded=df_branded.loc[df_branded["後発品区分"]==0]
df_generic=df_generic.loc[df_generic["後発品区分"]==1]
df_branded=df_branded.set_index(["id","year"])
df_generic=df_generic.set_index(["id","year"])
df=df.set_index(["id","year"])
df_branded=PanelData(df_branded,convert_dummies=False)
df_generic=PanelData(df_generic,convert_dummies=False)
df=PanelData(df,convert_dummies=False)

In [67]:
# df_year=df.dataframe.reset_index()
# df_year["year2"]=df_year["year"]
# df_year=df_year.set_index(["id","year"])
# df_year=PanelData(df_year,convert_dummies=False)

In [68]:
# all data
formula="log_quantity~ year2+後発品区分+oral+in_hospital+elasped_m5+elasped_m4+elasped_m3+elasped_m2+elasped_m1+elasped_0+elasped_1+elasped_2+elasped_3+elasped_4+elasped_5+elasped_6+elasped_7+elasped_8+elasped_9+elasped_10+elasped_11+elasped_12+elasped_13+elasped_14+elasped_15+TimeEffects+EntityEffects"
mod_fe=PanelOLS.from_formula(formula, data=df,check_rank=False,drop_absorbed=True)
res_fe=mod_fe.fit(cov_type='clustered', cluster_entity=True)
print(res_fe.summary.tables[1])
# print(res_fe.summary.tables[0])
# print(res_fe)

                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
year2           4.5682     0.1410     32.401     0.0000      4.2917      4.8448
後発品区分          -0.5288     0.2898    -1.8247     0.0682     -1.0973      0.0396
oral            3.0332     0.7966     3.8076     0.0001      1.4707      4.5957
in_hospital     0.1207     0.1611     0.7490     0.4540     -0.1953      0.4366
elasped_m5     -0.0578     0.0923    -0.6262     0.5313     -0.2388      0.1232
elasped_m4      0.1041     0.0845     1.2316     0.2183     -0.0617      0.2699
elasped_m3     -0.0236     0.1050    -0.2253     0.8218     -0.2295      0.1822
elasped_m2      0.1445     0.0972     1.4861     0.1374     -0.0462      0.3351
elasped_m1      0.1137     0.1280     0.8883     0.3745     -0.1374      0.3648
elasped_0       0.0504     0.1351     0.

In [69]:
# generic data
formula="log_quantity~oral+in_hospital+year2+generic_per+elasped_m5+elasped_m4+elasped_m3+elasped_m2+elasped_m1+elasped_0+elasped_1+elasped_2+elasped_3+elasped_4+elasped_5+elasped_6+elasped_7+elasped_8+elasped_9+elasped_10+TimeEffects+EntityEffects"
mod_fe=PanelOLS.from_formula(formula, data=df_generic,check_rank=False,drop_absorbed=True)
res_fe=mod_fe.fit(cov_type='clustered', cluster_entity=True)
# print(res_fe.summary.tables[1])
# print(res_fe.summary.tables[0])
print(res_fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:           log_quantity   R-squared:                        0.3426
Estimator:                   PanelOLS   R-squared (Between):             -0.7043
No. Observations:                 747   R-squared (Within):              -56.365
Date:                Thu, Aug 17 2023   R-squared (Overall):             -12.081
Time:                        21:01:00   Log-likelihood                   -1107.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      19.685
Entities:                          42   P-value                           0.0000
Avg Obs:                       17.786   Distribution:                  F(18,680)
Min Obs:                       6.0000                                           
Max Obs:                       32.000   F-statistic (robust):          8.647e+07
                            

C:\Users\hakos\AppData\Local\Temp\ipykernel_16544\2744887352.py:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

generic_per

  res_fe=mod_fe.fit(cov_type='clustered', cluster_entity=True)


In [71]:
# branded data
formula="log_quantity~ oral+in_hospital+year2+generic_per+elasped_m5+elasped_m4+elasped_m3+elasped_m2+elasped_m1+elasped_0+elasped_1+elasped_2+elasped_3+elasped_4+elasped_5+elasped_6+elasped_7+elasped_8+elasped_9+elasped_10+TimeEffects+EntityEffects"
mod_fe=PanelOLS.from_formula(formula, data=df_branded,check_rank=False,drop_absorbed=True)
res_fe=mod_fe.fit(cov_type='clustered', cluster_entity=True)
# print(res_fe.summary.tables[1])
# print(res_fe.summary.tables[0])
print(res_fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:           log_quantity   R-squared:                        0.3324
Estimator:                   PanelOLS   R-squared (Between):             -0.4134
No. Observations:                 940   R-squared (Within):              -21.847
Date:                Thu, Aug 17 2023   R-squared (Overall):             -11.256
Time:                        21:02:23   Log-likelihood                   -1875.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      24.116
Entities:                          43   P-value                           0.0000
Avg Obs:                       21.860   Distribution:                  F(18,872)
Min Obs:                       14.000                                           
Max Obs:                       32.000   F-statistic (robust):          4.189e+07
                            

C:\Users\hakos\AppData\Local\Temp\ipykernel_16544\2934924666.py:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

generic_per

  res_fe=mod_fe.fit(cov_type='clustered', cluster_entity=True)
